In [59]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat
import os
import pandas as pd
import tqdm
import re
import ast
import numpy as np
import time
import joblib
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score,\
                            f1_score, precision_score, recall_score, average_precision_score

In [2]:
path = 'new_books_prepared.csv'

df = pd.read_csv(path, index_col=0)
df['tokens'] = df.tokens.apply(ast.literal_eval)
df['labels'] = df.labels.apply(ast.literal_eval)
df = df[df.tokens.apply(len) < 200]
df_train, df_val_test = train_test_split(df, test_size=0.2, random_state=999)
df_val, df_test  = train_test_split(df_val_test, test_size=0.5, random_state=999)

df_test

,text,tokens,clear_punct_lower,labels
1341,Все это смог я различить лишь смутно и с трудо...,"[все, это, смог, я, различить, лишь, смутно, и...",все это смог я различить лишь смутно и с трудо...,"[o, o, o, o, o, o, o, o, o, ., o, o, o, o, o, ..."
25027,"Она поехала в игрушечную лавку, накупила игруш...","[она, поехала, в, игрушечную, лавку, накупила,...",она поехала в игрушечную лавку накупила игруше...,"[o, o, o, o, ,, o, o, o, o, o, ., o, o, o, ,, ..."
2585,Наконец настало утро четырнадцатого числа. пог...,"[наконец, настало, утро, четырнадцатого, числа...",наконец настало утро четырнадцатого числа пого...,"[o, o, o, o, ., o, o, o, o, o, o, o, ,, o, o, ..."
16829,"Хорошо. А почему прежде, бывало, с восьми часо...","[хорошо, а, почему, прежде, бывало, с, восьми,...",хорошо а почему прежде бывало с восьми часов в...,"[., o, o, ,, ,, o, o, o, o, o, o, o, ,, o, o, ..."
7937,"Говоря это, графиня оглянулась на дочь. Наташа...","[говоря, это, графиня, оглянулась, на, дочь, н...",говоря это графиня оглянулась на дочь наташа л...,"[o, ,, o, o, o, ., o, ,, o, o, o, o, o, o, o, ..."
...,...,...,...,...
13908,Разве на одну секунду... Я пришел за советом. ...,"[разве, на, одну, секунду, я, пришел, за, сове...",разве на одну секунду я пришел за советом я ко...,"[o, o, o, ..., o, o, o, ., ,, ,, o, o, o, ,, ,..."
21490,"План был очень хорош, но дело заключалось в то...","[план, был, очень, хорош, но, дело, заключалос...",план был очень хорош но дело заключалось в том...,"[o, o, o, ,, o, o, o, o, ,, o, o, o, o, o, o, ..."
2567,"Сохраняя, поелику возможно, равновесие, чтобы ...","[сохраняя, поелику, возможно, равновесие, чтоб...",сохраняя поелику возможно равновесие чтобы хор...,"[,, o, ,, ,, o, o, o, ,, o, o, ,, o, o, o, o, ..."
25405,"Было ли в лице Левина что-нибудь особенное, ил...","[было, ли, в, лице, левина, чтонибудь, особенн...",было ли в лице левина чтонибудь особенное или ...,"[o, o, o, o, o, o, ,, o, o, o, ,, o, o, o, o, ..."


In [7]:
GIGACHAT_TOKEN = 'NDIxMjA2NDktM2VjNS00ZmY0LWExNzItNzA2MTA3YzE4ODljOjMzNGY1OWE1LWU4ZGYtNDQ2Yi1iZDI5LTQ0YzQ4YjkyNGY0Mg==' # os.environ.get('GIGACHAT_TOKEN')

In [10]:
chat = GigaChat(credentials=GIGACHAT_TOKEN,
                    verify_ssl_certs=False)

chat.invoke('привет как дела').content

'Привет! Как дела?'

In [51]:
def gigachat_pred(text):
    chat = GigaChat(credentials=GIGACHAT_TOKEN,
                    verify_ssl_certs=False)

    messages = [SystemMessage(content="Правильно расставляй в тексте знаки препинания. Сохраняй количество слов в тексте"),
                HumanMessage(content='Текст: \n' + text)]
    answer = chat.invoke(messages).content
#     with open('gigachat_preds_test.txt', 'a') as file:
#         file.write(answer + '\n')
#         file.close()

    return answer

In [54]:
punctuation_signs = ['!', ',', '.', '...', ':', '?']

def prepare_pred(text):
    prediction = gigachat_pred(text)
    tokens = [token for token in prediction.split(' ') if token != '']
    labels = []
    
    for token in tokens:
        if (len(token) > 3) & (token[-3:] == '...'):
            labels.append('...')
        elif token[-1] in punctuation_signs:
            labels.append(token[-1])
        else:
            labels.append('o')
    
    return labels

prepare_pred('привет как у тебя дела')

[',', 'o', 'o', 'o', '?']

In [55]:
preds = []
true_labels = []

# процесс падал, поэтому продолжал генерировать с момента падения
for id_text in tqdm.tqdm(range(len(df_test.text.values))):
    time.sleep(3)
    prediction = prepare_pred(df_test.clear_punct_lower.values[id_text])   
    needed_labels = df_test.labels.values[id_text]
    
    # если модель выдала что-то не то по размеру
    if len(prediction) != len(needed_labels):        
        not_empty_token_idxs = ~(np.array(df_test.tokens.values[id_text]) == '')
        needed_labels = np.array(needed_labels)[not_empty_token_idxs].tolist()
        
    if len(needed_labels) == len(prediction):
        true_labels += needed_labels
        preds += prediction


100%|█████████████████████████████████████| 2586/2586 [3:15:46<00:00,  4.54s/it]


In [56]:
len(true_labels), len(preds)

(68141, 68141)

In [58]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y_pred = le.fit_transform(preds)
y_true = le.transform(true_labels)

y_true

array([6, 6, 6, ..., 6, 6, 2])

In [60]:
def calc_metrics_no_proba(y_true, y_pred):
    print('Доля пробелов:', (y_true == 6).mean())
    print('Accuracy:', accuracy_score(y_true, y_pred, ))
#     print('Top-2 Accuracy:', top_k_accuracy_score(y_true, y_pred_proba, k=2))
#     rint('ROC-AUC (OVR):',roc_auc_score(y_true, y_pred_proba, multi_class='ovr'))
#     print('AUC-PR:',average_precision_score(y_true, y_pred_proba, average='weighted'))
    
    metrics = []
    metrics.append(list(dict(sorted(Counter(y_true).items())).values()))
    metrics.append(f1_score(y_true, y_pred, average=None))
    metrics.append(precision_score(y_true, y_pred, average=None, zero_division=0))
    metrics.append(recall_score(y_true, y_pred, average=None, zero_division=0))
#     metrics.append(roc_auc_score(y_true, y_pred_proba, multi_class='ovr', average=None))
#     metrics.append(average_precision_score(y_true, y_pred_proba, average=None))
    metrics_index = ['Count', 'F1-Score', 'Precision', 'Recall']
#                      'ROC-AUC', 'AUC-PR']
    df_metrics = pd.DataFrame(metrics, columns=le.classes_, index=metrics_index)
    
    return df_metrics

In [61]:
calc_metrics_no_proba(y_true, y_pred)

Доля пробелов: 0.7968183619260064
Accuracy: 0.9298513376674836


,!,",",.,...,:,?,o
Count,127.000000,9061.000000,4222.000000,91.0,133.000000,211.000000,54296.000000
F1-Score,0.109589,0.787717,0.746078,0.0,0.190476,0.627848,0.969944
Precision,0.421053,0.792691,0.850303,0.0,0.321429,0.673913,0.958297
Recall,0.062992,0.782805,0.664614,0.0,0.135338,0.587678,0.981877


In [62]:
calc_metrics_no_proba(y_true, y_pred).to_excel('metrics_giga.xlsx')

Доля пробелов: 0.7968183619260064
Accuracy: 0.9298513376674836
